In [12]:
import os
import pandas as pd
print(os.getcwd())
for i in os.listdir():
    print(str(os.path.join(os.getcwd(),i)))

for i in os.listdir():
    if str(i).endswith(".csv"):
        print(pd.read_csv(i).columns)

l=[r"c:\Users\KIIT\Desktop\GitHUB\Assignments\soulpageit\test",
r"c:\Users\KIIT\Desktop\GitHUB\Assignments\soulpageit\license_plates_detection_train",
r"c:\Users\KIIT\Desktop\GitHUB\Assignments\soulpageit\license_plates_recognition_train"]
for i in l:
    print(len(os.listdir(path=i)))

c:\Users\KIIT\Desktop\GitHUB\Assignments\soulpageit
c:\Users\KIIT\Desktop\GitHUB\Assignments\soulpageit\License Plate Recognition.docx
c:\Users\KIIT\Desktop\GitHUB\Assignments\soulpageit\license_plates_detection_train
c:\Users\KIIT\Desktop\GitHUB\Assignments\soulpageit\license_plates_recognition_train
c:\Users\KIIT\Desktop\GitHUB\Assignments\soulpageit\Licplatesdetection_train.csv
c:\Users\KIIT\Desktop\GitHUB\Assignments\soulpageit\Licplatesrecognition_train.csv
c:\Users\KIIT\Desktop\GitHUB\Assignments\soulpageit\Main.ipynb
c:\Users\KIIT\Desktop\GitHUB\Assignments\soulpageit\SampleSubmission.csv
c:\Users\KIIT\Desktop\GitHUB\Assignments\soulpageit\test
c:\Users\KIIT\Desktop\GitHUB\Assignments\soulpageit\~$cense Plate Recognition.docx
Index(['img_id', 'ymin', 'xmin', 'ymax', 'xmax'], dtype='object')
Index(['img_id', 'text'], dtype='object')
Index(['id', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], dtype='object')
1
900
900


In [2]:
import os
import cv2
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import pickle

# === PATH SETUP ===
base_path = r"c:\Users\KIIT\Desktop\GitHUB\Assignments\soulpageit"
detect_img_dir = os.path.join(base_path, "license_plates_detection_train")
recog_img_dir = os.path.join(base_path, "license_plates_recognition_train")
test_img_dir = os.path.join(base_path, "test")

detect_csv_path = os.path.join(base_path, "Licplatesdetection_train.csv")
recog_csv_path = os.path.join(base_path, "Licplatesrecognition_train.csv")

# New output directory for saving models and submission
output_dir = os.path.join(base_path, "output")
os.makedirs(output_dir, exist_ok=True)

submission_path = os.path.join(output_dir, "SampleSubmission.csv")

# === TRAINING RECOGNITION MODEL ===
df_recog = pd.read_csv(recog_csv_path)

X, y = [], []
print("📦 Loading and preprocessing recognition images...")
for _, row in tqdm(df_recog.iterrows(), total=len(df_recog), desc="Loading images"):
    img_path = os.path.join(recog_img_dir, row['img_id'])
    if os.path.exists(img_path):
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (128, 64))  # Width=128, Height=64
        X.append(img)
        y.append(row['text'])
    else:
        print(f"⚠️ Image not found: {img_path}")

X = np.array(X).reshape(-1, 64, 128, 1) / 255.0
y = np.array(y)

# Encode labels (whole plate text classification)
print("🔤 Encoding plate text labels...")
label_encoder = LabelBinarizer()
y_encoded = label_encoder.fit_transform(y)

# Split dataset
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.1, random_state=42)

# Build CNN model
print("🧠 Building CNN model...")
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(64, 128, 1)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train model with 100 epochs
print("🎯 Training CNN model...")
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=32)

# Save model and label encoder to new output folder
model_save_path = os.path.join(output_dir, "lp_recognition_model.h5")
label_encoder_path = os.path.join(output_dir, "label_encoder.pkl")

model.save(model_save_path)
with open(label_encoder_path, "wb") as f:
    pickle.dump(label_encoder, f)

print(f"✅ Model training complete and saved to:\n  - {model_save_path}\n  - {label_encoder_path}")

# === TESTING & PREDICTION ===
print("\n🧪 Running on test set...")
test_img_files = sorted(os.listdir(test_img_dir))
predictions = []

# Load model and encoder from new output folder
model = load_model(model_save_path)
with open(label_encoder_path, "rb") as f:
    label_encoder = pickle.load(f)

print("🔍 Detecting and recognizing license plates on test images...")
for img_file in tqdm(test_img_files, desc="Test images"):
    img_path = os.path.join(test_img_dir, img_file)
    img = cv2.imread(img_path)
    if img is None:
        # If image loading failed
        predictions.append([img_file] + [""] * 10)  # Pad 10 columns for submission
        continue

    h, w = img.shape[:2]
    cx, cy = w // 2, h // 2
    dw, dh = int(w * 0.3), int(h * 0.15)  # 60% crop width and 30% crop height approx.
    xmin, xmax = max(0, cx - dw), min(w, cx + dw)
    ymin, ymax = max(0, cy - dh), min(h, cy + dh)
    plate_crop = img[ymin:ymax, xmin:xmax]

    # Save cropped plate image to output folder as JPG
    plate_img_save_path = os.path.join(output_dir, f"{os.path.splitext(img_file)[0]}_plate.jpg")
    cv2.imwrite(plate_img_save_path, plate_crop)

    try:
        plate_crop_gray = cv2.cvtColor(plate_crop, cv2.COLOR_BGR2GRAY)
        plate_crop_gray = cv2.resize(plate_crop_gray, (128, 64))
        plate_crop_gray = plate_crop_gray.reshape(1, 64, 128, 1) / 255.0

        # Suppress keras verbose output here for clean progress bar
        pred = model.predict(plate_crop_gray, verbose=0)
        pred_label = label_encoder.inverse_transform(pred)[0]

        # Pad with space ' ' which is exactly one character
        padded = list(pred_label.ljust(10, ' '))
        predictions.append([img_file] + padded[:10])
    except Exception as e:
        tqdm.write(f"⚠️ Prediction error on {img_file}: {e}")
        predictions.append([img_file] + [""] * 10)

# Save submission CSV in new output folder
df_sub = pd.DataFrame(predictions, columns=['id'] + [str(i) for i in range(10)])
df_sub.to_csv(submission_path, index=False)
print(f"📄 Submission saved to: {submission_path}")


📦 Loading and preprocessing recognition images...


Loading images:   0%|          | 0/900 [00:00<?, ?it/s]

Loading images: 100%|██████████| 900/900 [00:01<00:00, 865.21it/s]


🔤 Encoding plate text labels...
🧠 Building CNN model...
🎯 Training CNN model...
Epoch 1/100
26/26 [==============================] - 9s 271ms/step - loss: 6.4006 - accuracy: 0.0012 - val_loss: 6.3974 - val_accuracy: 0.0111
Epoch 2/100
26/26 [==============================] - 6s 249ms/step - loss: 6.3878 - accuracy: 0.0074 - val_loss: 6.4103 - val_accuracy: 0.0111
Epoch 3/100
26/26 [==============================] - 6s 250ms/step - loss: 6.3674 - accuracy: 0.0074 - val_loss: 6.4872 - val_accuracy: 0.0111
Epoch 4/100
26/26 [==============================] - 7s 268ms/step - loss: 6.3220 - accuracy: 0.0049 - val_loss: 6.5950 - val_accuracy: 0.0111
Epoch 5/100
26/26 [==============================] - 7s 262ms/step - loss: 6.2394 - accuracy: 0.0148 - val_loss: 6.7902 - val_accuracy: 0.0000e+00
Epoch 6/100
26/26 [==============================] - 7s 253ms/step - loss: 6.1343 - accuracy: 0.0247 - val_loss: 7.0913 - val_accuracy: 0.0111
Epoch 7/100
26/26 [==============================] - 7s 25

Test images: 100%|██████████| 210/210 [00:17<00:00, 12.34it/s]

📄 Submission saved to: c:\Users\KIIT\Desktop\GitHUB\Assignments\soulpageit\output\SampleSubmission.csv


In [3]:
# === IMPORT LIBRARIES ===
import os
import cv2
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import pickle

# === PATH SETUP ===
# Base directory for all input/output files
base_path = r"c:\Users\KIIT\Desktop\GitHUB\Assignments\soulpageit"

# Training data directories
detect_img_dir = os.path.join(base_path, "license_plates_detection_train")
recog_img_dir = os.path.join(base_path, "license_plates_recognition_train")
test_img_dir = os.path.join(base_path, "test")

# CSV annotation files
detect_csv_path = os.path.join(base_path, "Licplatesdetection_train.csv")
recog_csv_path = os.path.join(base_path, "Licplatesrecognition_train.csv")

# Output directory for saving model, encoder, and submission
output_dir = os.path.join(base_path, "output")
os.makedirs(output_dir, exist_ok=True)  # Create if not exists

# Output submission file path
submission_path = os.path.join(output_dir, "SampleSubmission.csv")

# === STEP 1: LOAD AND PREPROCESS RECOGNITION DATA ===
df_recog = pd.read_csv(recog_csv_path)

X, y = [], []
print("📦 Loading and preprocessing recognition images...")
for _, row in tqdm(df_recog.iterrows(), total=len(df_recog), desc="Loading images"):
    img_path = os.path.join(recog_img_dir, row['img_id'])
    if os.path.exists(img_path):
        # Load image in grayscale
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (128, 64))  # Resize to fixed dimensions
        X.append(img)
        y.append(row['text'])  # Plate text
    else:
        print(f"⚠️ Image not found: {img_path}")

# Convert to numpy array and normalize
X = np.array(X).reshape(-1, 64, 128, 1) / 255.0  # Normalize to [0,1]
y = np.array(y)

# === STEP 2: ENCODE TEXT LABELS ===
print("🔤 Encoding plate text labels...")
label_encoder = LabelBinarizer()
y_encoded = label_encoder.fit_transform(y)  # One-hot encoding for plate strings

# === STEP 3: SPLIT TRAIN/VALIDATION DATA ===
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.1, random_state=42)

# === STEP 4: DEFINE CNN MODEL ===
print("🧠 Building CNN model...")
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(64, 128, 1)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),  # Prevent overfitting
    Dense(len(label_encoder.classes_), activation='softmax')  # Output layer
])
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# === STEP 5: TRAIN MODEL ===
print("🎯 Training CNN model...")
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=32)

# === STEP 6: SAVE MODEL AND LABEL ENCODER ===
model_save_path = os.path.join(output_dir, "lp_recognition_model.h5")
label_encoder_path = os.path.join(output_dir, "label_encoder.pkl")

model.save(model_save_path)
with open(label_encoder_path, "wb") as f:
    pickle.dump(label_encoder, f)

print(f"✅ Model training complete and saved to:\n  - {model_save_path}\n  - {label_encoder_path}")

# === STEP 7: RUN ON TEST SET ===
print("\n🧪 Running on test set...")
test_img_files = sorted(os.listdir(test_img_dir))
predictions = []

# Load saved model and encoder
model = load_model(model_save_path)
with open(label_encoder_path, "rb") as f:
    label_encoder = pickle.load(f)

print("🔍 Detecting and recognizing license plates on test images...")
for img_file in tqdm(test_img_files, desc="Test images"):
    img_path = os.path.join(test_img_dir, img_file)
    img = cv2.imread(img_path)

    if img is None:
        # Could not load image; fill with blanks
        predictions.append([img_file] + [""] * 10)
        continue

    # Simple plate localization: center crop
    h, w = img.shape[:2]
    cx, cy = w // 2, h // 2
    dw, dh = int(w * 0.3), int(h * 0.15)  # Approx crop box
    xmin, xmax = max(0, cx - dw), min(w, cx + dw)
    ymin, ymax = max(0, cy - dh), min(h, cy + dh)
    plate_crop = img[ymin:ymax, xmin:xmax]

    # Save cropped plate for visual inspection
    plate_img_save_path = os.path.join(output_dir, f"{os.path.splitext(img_file)[0]}_plate.jpg")
    cv2.imwrite(plate_img_save_path, plate_crop)

    try:
        # Preprocess cropped plate image
        plate_crop_gray = cv2.cvtColor(plate_crop, cv2.COLOR_BGR2GRAY)
        plate_crop_gray = cv2.resize(plate_crop_gray, (128, 64))
        plate_crop_gray = plate_crop_gray.reshape(1, 64, 128, 1) / 255.0

        # Predict plate text
        pred = model.predict(plate_crop_gray, verbose=0)
        pred_label = label_encoder.inverse_transform(pred)[0]

        # Pad prediction to 10 characters
        padded = list(pred_label.ljust(10, ' '))
        predictions.append([img_file] + padded[:10])
    except Exception as e:
        tqdm.write(f"⚠️ Prediction error on {img_file}: {e}")
        predictions.append([img_file] + [""] * 10)

# === STEP 8: SAVE SUBMISSION FILE ===
df_sub = pd.DataFrame(predictions, columns=['id'] + [str(i) for i in range(10)])
df_sub.to_csv(submission_path, index=False)
print(f"📄 Submission saved to: {submission_path}")


📦 Loading and preprocessing recognition images...


Loading images: 100%|██████████| 900/900 [00:00<00:00, 965.47it/s] 


🔤 Encoding plate text labels...
🧠 Building CNN model...
🎯 Training CNN model...
Epoch 1/100
26/26 [==============================] - 7s 242ms/step - loss: 6.4109 - accuracy: 0.0037 - val_loss: 6.3970 - val_accuracy: 0.0000e+00
Epoch 2/100
26/26 [==============================] - 7s 263ms/step - loss: 6.3767 - accuracy: 0.0099 - val_loss: 6.4259 - val_accuracy: 0.0000e+00
Epoch 3/100
26/26 [==============================] - 7s 285ms/step - loss: 6.3337 - accuracy: 0.0123 - val_loss: 6.7218 - val_accuracy: 0.0000e+00
Epoch 4/100
26/26 [==============================] - 7s 254ms/step - loss: 6.2462 - accuracy: 0.0160 - val_loss: 6.5622 - val_accuracy: 0.0111
Epoch 5/100
26/26 [==============================] - 7s 252ms/step - loss: 6.1036 - accuracy: 0.0173 - val_loss: 7.0177 - val_accuracy: 0.0111
Epoch 6/100
26/26 [==============================] - 6s 249ms/step - loss: 5.8742 - accuracy: 0.0370 - val_loss: 6.8849 - val_accuracy: 0.0111
Epoch 7/100
26/26 [==============================]

Test images: 100%|██████████| 210/210 [00:17<00:00, 12.26it/s]

📄 Submission saved to: c:\Users\KIIT\Desktop\GitHUB\Assignments\soulpageit\output\SampleSubmission.csv
